<a href="https://colab.research.google.com/github/rampopat/uncertainty/blob/main/Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performance Analysis

This notebook contains the performance analysis for both datasets. Broadly, we have the imports then the key performance analysis.

# Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, roc_curve, plot_roc_curve, confusion_matrix, classification_report
from sklearn.calibration import calibration_curve
import sklearn.metrics
from scipy.stats import entropy

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
!pip install netcal
import netcal as nc

In [ ]:
from netcal.scaling import TemperatureScaling
from netcal.metrics import ECE
from netcal.presentation import ReliabilityDiagram

In [ ]:
import warnings
import sklearn
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [ ]:
%load_ext google.colab.data_table

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
!unzip drive/MyDrive/UncertaintyAnalysisData.zip

# Performance Analysis

In [ ]:
def binarize(ps, threshold=0.5):
    return np.where(ps > threshold, 1, 0)

def ece(y_true, y_pred, bins=30):
    # This is a pmetric because we want to calculate this on portions of data rather than order data by it
    # Use same function signature as the other pmetrics to allow usage
    ece = ECE(bins)
    return 1 - ece.measure(y_pred, y_true)

def ece_ts(y_true, y_pred, bins=30):
    # This is a pmetric because we want to calculate this on portions of data rather than order data by it
    # Use same function signature as the other pmetrics to allow usage
    temperature = TemperatureScaling()
    temperature.fit(y_pred, y_true)
    calibrated = temperature.transform(y_pred)
    ece = ECE(bins)
    return 1 - ece.measure(calibrated, y_true)

PATH = 'UncertaintyAnalysisData/'  

def get_df(name1, name2):    
    df = pd.read_pickle(PATH + name1).sort_index()
    all_preds = pd.read_pickle(PATH + name2).to_numpy()
    return df, all_preds
def get_df_from_preds(all_preds=None, name=None, labels=None, threshold=0.5):
    if all_preds is None:
        all_preds = pd.read_pickle(PATH + name).to_numpy()    
    # all_preds.shape = (ensemble_size, data_size)
    preds = all_preds.mean(axis=0)
    df = pd.DataFrame()
    df['pred'] = preds
    df['pred_label'] = binarize(preds, threshold=threshold)
    df['true_label'] = labels
    df['correct'] = df['pred_label'] == df['true_label']
    df = df.astype({'correct': int })
    return df, all_preds

In [ ]:
preds_rnn_mc = np.load(PATH + 'rnnmc.npy')
labels = get_df('rnnens10.pkl', 'rnnens10ap.pkl')[0].true_label
df_rnn_mc, preds_rnn_mc = get_df_from_preds(all_preds=np.load(PATH + 'rnnmc.npy'), labels=labels)
df_rnn, preds_rnn = get_df_from_preds(name='rnnens10ap.pkl', labels=labels)
# df_trf, preds_trf = get_df_from_preds(name='trf10ap.pkl', labels=labels) #tbert
df_brnns, preds_brnns = get_df_from_preds(name='brnnens5ap.pkl', labels=labels)
df_brnn, preds_brnn = get_df_from_preds(name='brnnap.pkl', labels=labels)
df_trf, preds_trf = get_df_from_preds(name='troberta5ap.pkl', labels=labels)
df_strf, preds_strf = get_df_from_preds(name='sroberta5ap.pkl', labels=labels)
df_trf_mc, preds_trf_mc = get_df_from_preds(name='trobertamcseed3.pkl', labels=labels)

some_dfs = [df_rnn, df_brnn, df_brnns, df_trf]
SOME_DF_NAMES = ['RNN-ENS', 'BRNN', 'BRNN-ENS', 'T-TRF-ENS']

dfs = [df_rnn_mc, df_rnn, df_brnn, df_brnns, df_trf_mc, df_trf, df_strf]
DF_NAMES = ['RNN-MC', 'RNN-ENS', 'BRNN', 'BRNN-ENS', 'T-TRF-MC', 'T-TRF-ENS', 'STRF-ENS']

preds_ens = [preds_rnn, preds_brnns, preds_trf, preds_strf]
dfs_ens = [df_rnn, df_brnns, df_trf, df_strf]
DF_NAMES_ENS = ['RNN-ENS', 'BRNN-ENS', 'T-TRF-ENS', 'S-TRF-ENS']

In [ ]:
patient_num = 19
bins=1000
brnn_preds = preds_brnns[:, patient_num]
rnn_preds = preds_rnn[:, patient_num]
plt.hist(brnn_preds, 10, density=True)
plt.hist(rnn_preds, 10, density=True)
plt.show()

In [ ]:
for i, preds in enumerate(preds_ens):
    print(DF_NAMES_ENS[i])
    print(preds.shape)

In [ ]:
pd.Series(np.rint(preds_rnn).mean(axis=0)).hist()

In [ ]:
for i, preds in enumerate(preds_ens):
    sns.histplot(pd.Series(np.rint(preds).mean(axis=0)))
    plt.xlabel('Mean of Ensemble Predicted Labels')
    plt.title(DF_NAMES_ENS[i])
    plt.show()

In [ ]:
rinted_preds = [np.rint(preds).mean(axis=0) for preds in preds_ens[:-1]]
plt.hist(rinted_preds, label=DF_NAMES_ENS[:-1])
plt.xlabel('Mean of Ensemble Predicted Labels')
plt.ylabel('Number of Patients')
plt.title('Disagreement Intra-Ensemble')
plt.legend()
plt.savefig('disagreement-ensemble', dpi=DPI)
plt.show()

In [ ]:
PMETRIC_FNS = [accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, ece, ece_ts]
PMETRIC_NAMES = ['Accuracy', 'F1', 'ROC AUC', 'Precision', 'Recall', '1-ECE', '1-ECE+TS'] 

In [ ]:
def compute_pmetrics(df, ret_cfm=False):
    true_labels = df.true_label.to_numpy()
    pred_labels = df.pred_label.to_numpy()
    preds = df.pred.to_numpy()
    results = []
    for metric in PMETRIC_FNS:
        if metric == roc_auc_score or metric == ece or metric == ece_ts:
            result = metric(true_labels, preds)
        else:
            result = metric(true_labels, pred_labels)
        results.append(result)
    if ret_cfm: 
        return confusion_matrix(true_labels, pred_labels, normalize='all')
    return results
    

In [ ]:
def compute_pmetrics_for_preds(ens_preds, labels):
    # ens_preds.shape = (ensemble_size, data_size)
    # rinted_preds = np.rint(ens_preds)
    true_labels = labels.to_numpy()
    results = []
    for i, preds in enumerate(ens_preds):
        # preds.shape = (data_size,)
        model_results = []
        for metric in PMETRIC_FNS:
            if metric == roc_auc_score or metric == ece or metric == ece_ts:
                result = metric(true_labels, preds)
            else:
                result = metric(true_labels, np.rint(preds))
            model_results.append(result)
        results.append(model_results.copy())
    return np.array(results) # shape = (ensemble_size, PMETRICS)

In [ ]:
def calibration_analysis(df):
    confidences = df.pred.to_numpy()
    ground_truth = df.true_label.to_numpy()
    temperature = TemperatureScaling()
    temperature.fit(confidences, ground_truth)
    calibrated = temperature.transform(confidences)

    n_bins = 30
    print('ECE Scores')
    ece = ECE(n_bins)
    uncalibrated_score = ece.measure(confidences, ground_truth)
    print('uncalibrated', uncalibrated_score)
    calibrated_score = ece.measure(calibrated, ground_truth)
    print('after temp scaling', calibrated_score)

    diagram = ReliabilityDiagram(n_bins)
    _ = diagram.plot(confidences, ground_truth) 
    _ = diagram.plot(calibrated, ground_truth) 

In [ ]:
calibration_analysis(df_rnn)

In [ ]:
pmetrics = compute_pmetrics_for_preds(preds_rnn, labels)
plt.scatter(np.arange(pmetrics.shape[0]), pmetrics[:, 0], label='Individual Scores')
plt.axhline(y=accuracy_score(labels.to_numpy(), np.rint(preds_rnn.mean(axis=0))), color='r', linestyle='-', label='Ensembled Score')
plt.axhline(y=pmetrics[:, 0].mean(axis=0), color='b', linestyle='-', label='Mean Score')
plt.xlabel('Model number')
plt.ylabel('Accuracy Score')
plt.legend(loc='best')
plt.show()

In [ ]:
metric_num = 0 # accuracy_score
print(PMETRIC_NAMES[metric_num]) # this is what we're getting
results = []
for i, preds in enumerate(preds_ens):
    # Get metrics for individual models in the ensemble
    # shape = (ensemble_size,)
    metrics_per_model = compute_pmetrics_for_preds(preds, labels)[:, metric_num]
    mean_metric = metrics_per_model.mean()
    std_metric = metrics_per_model.std()
    mean_pm_str = '$' + str(mean_metric.round(3)) + 'pm' + str(std_metric.round(3)) + '$'
    print(mean_pm_str)
    ensembled_score = compute_pmetrics(dfs_ens[i])[metric_num]
    results.append([ensembled_score, mean_pm_str, metrics_per_model.min(), metrics_per_model.max()])
ens_results = pd.DataFrame(results, columns=['Ensembled', 'Mean $\pm$ Std', 'Min', 'Max']).round(3)
ens_results.index = DF_NAMES_ENS
ens_results

In [ ]:
print(ens_results.to_latex()) # it's a bit messed up 

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
p_results = [compute_pmetrics(df) for df in dfs]
# shape = [num_dfs, num_pmetrics]
p_results = np.array(p_results).round(3)
p_results = pd.DataFrame(p_results, columns=PMETRIC_NAMES)
p_results.index = DF_NAMES
p_results

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
p_results = [compute_pmetrics(all_preds) for all_preds in preds_ens]
# shape = [num_dfs, num_pmetrics]
p_results = np.array(p_results).round(3)
p_results = pd.DataFrame(p_results, columns=PMETRIC_NAMES)
p_results.index = DF_NAMES
p_results

In [ ]:
print(p_results.to_latex())

In [ ]:
import re
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
p_results = np.array([compute_pmetrics_for_preds(ps, labels).mean(axis=0) for ps in preds_ens]).round(3)
# shape = [num_dfs, num_pmetrics]
p_results = pd.DataFrame(p_results, columns=PMETRIC_NAMES)
p_results.index = [re.sub('-ENS', '', name) for name in DF_NAMES_ENS]
print(p_results.to_latex())

In [ ]:
cfms = [compute_pmetrics(df, ret_cfm=True) for df in some_dfs]

In [ ]:
print(np.array(cfms))

In [ ]:
plt.subplots(1, 4, figsize=(8, 2))
plt.tight_layout()
for i in range(len(cfms)):
    plt.subplot(1, 4, i + 1)
    make_confusion_matrix(cfms[i], count=False, categories=['HC', 'AD'], sum_stats=False, cbar=False, 
                          title=SOME_DF_NAMES[i])
plt.tight_layout()
plt.savefig('confusion-matrices', dpi=DPI)
plt.show()

In [ ]:
def get_roc_curve(df, name=''):
    fpr, tpr, thresholds = roc_curve(df.true_label, df.pred)
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    label = name + ' (AUC = {:.2f})'.format(roc_auc)
    # display = sklearn.metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=None)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.plot(fpr, tpr, label=label) 
    plt.legend()    

In [ ]:
plt.figure(figsize=(5, 5))   
for i in range(len(dfs)):
    get_roc_curve(dfs[i], name=DF_NAMES[i])
plt.title('ROC Curves')
plt.savefig('roc-curves', dpi=DPI)
plt.show() 

In [ ]:
def get_calibration_curve(df, name=' ', bins=10):
    true_labels = df.true_label.to_numpy()
    preds = df.pred.to_numpy()
    y, x = calibration_curve(true_labels, preds, n_bins=bins)
    plt.plot(x, y, label=name, marker='.')


In [ ]:
bins = 5
plt.figure(figsize=(8,5))   
plt.plot([0, 1], [0, 1], "k:", label="Perfectly Calibrated")
for i in range(len(dfs)):
    get_calibration_curve(dfs[i], name=DF_NAMES[i], bins=bins)
plt.title('Reliability Diagrams')
plt.xlabel('Predicted Probability')
plt.ylabel('Observed Probability')
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
# plt.legend()
plt.tight_layout()
plt.rcParams.update({'font.size': 16})
plt.savefig('reliability-diagrams', dpi=DPI)
plt.show() 